## 9. 임베딩 내 편향성 알아보기 - Project

### WEAT 구현하기

In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [2]:
def weat_score(X, Y, A, B):
    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

### 직접 만드는 Word Embedding에 WEAT 적용

#### 1. 형태소 분석기를 이용하여 품사가 명사인 경우 해당 단어를 추출하기

In [3]:
import os

with open('./weat/synopsis.txt', 'r') as file:
    for i in range(20):
        print(file.readline(), end='')

사운드 엔지니어 상우(유지태 분)는 치매에 걸린 할머니(백성희 분)와
 젊은 시절 상처한 한 아버지(박인환 분), 고모(신신애 분)와 함께 살고 있다.
 어느 겨울 그는 지방 방송국 라디오 PD 은수(이영애 분)를 만난다.
 자연의 소리를 채집해 틀어주는 라디오 프로그램을 준비하는 은수는 상우와 녹음 여행을 떠난다.
 자연스레 가까워지는 두 사람은 어느 날, 은수의 아파트에서 밤을 보낸다.
 너무 쉽게 사랑에 빠진 두 사람... 상우는 주체할 수 없을 정도로 그녀에게 빨려든다.
 그러나 겨울에 만난 두 사람의 관계는 봄을 지나 여름을 맞이하면서 삐걱거린다.
 이혼 경험이 있는 은수는 상우에게 결혼할 생각이 없다며 부담스러운 표정을 내비친다.
 "어떻게 사랑이 변하니?..."라고 묻는 상우에게 은수는 그저 "헤어져" 라고 단호하게 말한다.
 영원히 변할 것 같지 않던 사랑이 변하고, 그 사실을 받아들이지 못하는 상우는 어찌 할 바를 모른다.
 은수를 잊지 못하는 상우는 미련과 집착의 감정을 이기지 못하고 서울과 강릉을 오간다.
유사 이래 연령, 성별, 빈부의 차이와 정치적인 입장을 불문하고 일거에 국민을 통합해 온 '애국심'이라는 성역에 일침을 가하는 다큐멘터리. 재작년 전국 민족민주 유가족협의회의 장기농성을 다룬 인상적인 다큐멘터리 <민들레>를 만들었던 독립영화집단 '빨간 눈사람'이 우리 사회 구석구석을 발빠르게 돌아다니며 애국심과 민족주의가 강요되는 현장을 발굴하여 카메라에 담았다. 박홍 서강대 명예총장, 이도형 '한국논단' 발행인, 축구해설자 신문선, 홍세화, 박노해 등 사회 각계의 '스타'들이 등장해 저마다의 확고한 신념을 성토한다. 감독 이경순과 최하동하는 이 작품을 위해 3년간 백여 명을 인터뷰했다고 한다. 2001 올해의 독립영화상 수상.
 민족과 국가란 공동체에서 부단히 권력과 부를 얻는 자, 나아가 민족과 국가란 공동체에서 얻은 신분과 부귀를 영원히 그의 자손에게 대물림하려는 자, 그래서 민족과 국가란 공동체를 부단히 유지해야만 하는 자, 따라서

In [4]:
# My code better
#약 15분정도 걸립니다.

from konlpy.tag import Okt
okt = Okt()
tokenized = []
with open('./weat/synopsis.txt', 'r') as file:
    while True:
        line = file.readline()
        if not line: break
        words = okt.pos(line, stem=True, norm=True) #norm : Normalize
        res = []
        for w in words:
            if "Noun" in w:      # "Adjective", "Verb" 등을 포함할 수도 있습니다.
                res.append(w[0])    # 명사일 때만 tokenized 에 저장하게 됩니다. 
                
        tokenized.append(res)

In [5]:
tokenized[:10]

[['사운드', '엔지니어', '상우', '유지태', '분', '치매', '할머니', '백성희', '분'],
 ['시절', '상처', '아버지', '박인환', '분', '고모', '신신애', '분', '살'],
 ['겨울', '그', '지방', '방송국', '라디오', '은수', '이영애', '분', '를'],
 ['자연', '소리', '채집', '라디오', '프로그램', '준비', '은수', '상우', '녹음', '여행'],
 ['두', '사람', '날', '은수', '아파트', '밤'],
 ['사랑', '두', '사람', '상우', '주체', '수', '정도', '그녀'],
 ['겨울', '만난', '두', '사람', '관계', '봄', '지나', '여름', '맞이', '삐걱', '거', '린다'],
 ['이혼', '경험', '은수', '상우', '결혼', '생각', '표정', '내비'],
 ['사랑', '상우', '은수', '단호', '말'],
 ['것', '사랑', '그', '사실', '상우', '어찌']]

#### 2. 추출된 결과로 embedding model 만들기

In [17]:
from gensim.models import Word2Vec

# tokenized에 담긴 데이터를 가지고 나만의 Word2Vec을 생성합니다. (Gensim 4.0 기준)
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)  
model.wv.most_similar(positive=['영화'])  #cosin 유사도 계산

[('작품', 0.8755549192428589),
 ('다큐멘터리', 0.8398740887641907),
 ('드라마', 0.8251179456710815),
 ('영화로', 0.7821416258811951),
 ('형식', 0.7816640138626099),
 ('주제', 0.7680657505989075),
 ('코미디', 0.7605226039886475),
 ('소재', 0.7600834965705872),
 ('감동', 0.7532719373703003),
 ('편의', 0.7438781261444092)]

#### 3. TF-IDF로 해당 데이터를 가장 잘 표현하는 단어 셋 만들기

In [18]:
## My code better

import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from konlpy.tag import Okt


import os

genre_txt = []
genre_names = []

directory = './weat/'
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        genre_txt.append(filename)

for filename in genre_txt:
    start = filename.find('_') + 1
    end = filename.find('.txt')
    result = filename[start:end]
    genre_names.append(result)


In [82]:
len(genre_names)

24

In [125]:
## My code better

def read_token(file_names):
    okt = Okt()
    document = []
   
    for file_name in file_names:
        result = []
        with open('./weat/'+ file_name, 'r') as fread: 
            print(file_name, '파일을 읽고 있습니다.')
            while True:
                line = fread.readline() 
                if not line: break 
                tokenlist = okt.pos(line, stem=True, norm=True) 
                for word in tokenlist:
                    if "Noun" in word:#, "Adjective", "Verb"]:
                        result.append((word[0])) 
            result = ' '.join(result)
        document.append(result)
        #print(document)
    return document

In [126]:
len(tokenized_files)

24

In [128]:
tokenized_files = read_token(genre_txt)

synopsis_family.txt 파일을 읽고 있습니다.
synopsis_horror.txt 파일을 읽고 있습니다.
synopsis_gen.txt 파일을 읽고 있습니다.
synopsis_comedy.txt 파일을 읽고 있습니다.
synopsis_art.txt 파일을 읽고 있습니다.
synopsis.txt 파일을 읽고 있습니다.
synopsis_mystery.txt 파일을 읽고 있습니다.
synopsis_adventure.txt 파일을 읽고 있습니다.
synopsis_documentary.txt 파일을 읽고 있습니다.
synopsis_romance.txt 파일을 읽고 있습니다.
synopsis_historical.txt 파일을 읽고 있습니다.
synopsis_thriller.txt 파일을 읽고 있습니다.
synopsis_fantasy.txt 파일을 읽고 있습니다.
synopsis_action.txt 파일을 읽고 있습니다.
synopsis_show.txt 파일을 읽고 있습니다.
synopsis_crime.txt 파일을 읽고 있습니다.
synopsis_musical.txt 파일을 읽고 있습니다.
synopsis_drama.txt 파일을 읽고 있습니다.
synopsis_adult.txt 파일을 읽고 있습니다.
synopsis_animation.txt 파일을 읽고 있습니다.
synopsis_war.txt 파일을 읽고 있습니다.
synopsis_western.txt 파일을 읽고 있습니다.
synopsis_etc.txt 파일을 읽고 있습니다.
synopsis_SF.txt 파일을 읽고 있습니다.


In [139]:
def tfidf(tokenized_files, genre_names):

    print(tokenized_files)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(tokenized_files)
    
    result = []
    for x, name in zip(X,genre_names) :
        m = x.tocoo()   # art를 TF-IDF로 표현한 sparse matrix를 가져옵니다. 
        #w = [[i, j] for i, j in zip(m.col, m.data)] #[col : column_index, data : value] 
        w = [[i, j] for i, j in zip(m.col, m.data)] #[col : column_index, data : value] 
        w.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 
        result.append(w)
        
        print(name+': ')
        for i in range(100):
            print(vectorizer.get_feature_names_out()[w[i][0]], end=', ')
        print('\n')
    
    return result

In [142]:
tfidf = tfidf(tokenized_files, genre_names)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



family: 
엄마, 아빠, 가족, 영화제, 자신, 위해, 친구, 아버지, 시작, 그녀, 아주르, 아들, 마을, 국제, 할머니, 낙타, 사람, 아이, 사랑, 학교, 씨제이, 세상, 동구, 소년, 이야기, 가장, 어머니, 슈이트, 소녀, 대한, 모두, 미아, 혼자, 서울, 사건, 마음, 엠마, 할아버지, 아내, 사이, 마갈, 펠리칸, 모험, 고양이, 크리스마스, 케이시, 위기, 요정, 벤트, 다시, 테리, 과연, 결심, 시골, 손녀, 작품, 영화, 유나, 번개, 최호, 혜지, 단편, 기억, 인형, 서로, 하나, 발견, 한편, 해나, 스튜어트, 부산, 도시, 그레고리, 아스, 장난감, 때문, 이자, 파코, 아기, 생각, 사실, 회사, 제노, 노동자, 다른, 시간, 소식, 간다, 마르, 강아지, 펠릭스, 통해, 도움, 모든, 주니, 크누드, 신차, 누키, 편지, 우산, 

horror: 
시작, 위해, 사람, 자신, 친구, 그녀, 사건, 공포, 발견, 죽음, 마을, 가족, 남자, 영화, 좀비, 악령, 사실, 소녀, 하나, 살인, 아이, 이야기, 비밀, 정체, 저주, 살해, 영화제, 여자, 모든, 점점, 의문, 명의, 사고, 시간, 다시, 이후, 엄마, 모두, 일행, 시체, 다른, 계속, 아버지, 악몽, 대한, 파티, 병원, 공격, 모습, 존재, 과연, 실종, 발생, 유령, 아들, 한편, 게임, 여행, 귀신, 악마, 충격, 저택, 아내, 사랑, 간다, 이사, 소리, 바이러스, 부부, 탈출, 학교, 과거, 인간, 현상, 영혼, 생각, 괴물, 사라, 로부터, 국제, 도시, 사이, 살인마, 상황, 학생, 동생, 순간, 판타스틱, 부천, 실험, 위협, 인형, 이제, 목숨, 자살, 갑자기, 기운, 통해, 도착, 무엇, 

gen: 
자신, 그녀, 영화제, 위해, 사람, 시작, 국제, 영화, 친구, 사랑, 남자, 이야기, 서울, 여자, 대한, 사건, 남편, 가족, 아이, 아버지, 단편, 엄마, 다른, 애니메이션, 마을, 모든, 시간, 마음, 사실, 다시, 아들, 모습, 작품,

# 4. embedding model과 단어 셋으로 WEAT score 구해보기

In [ ]:
n = len(genre_name)
matrix = np.zeros((n, n))
#matrix= [[]]

In [ ]:
X = np.array([model.wv[word] for word in target_art])
Y = np.array([model.wv[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model.wv[word] for word in attributes[i]])
        B = np.array([model.wv[word] for word in attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)

In [ ]:
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print('['+ genre_name[i], genre_name[j]+']',matrix[i][j])

from matplotlib import font_manager
fonts = [font.name for font in font_manager.fontManager.ttflist]
print(fonts)

In [ ]:
import numpy as np; 
import seaborn as sns; 

np.random.seed(0)

# 한글 지원 폰트
sns.set(font='NanumGothic')

# 마이너스 부호 

plt.rcParams['axes.unicode_minus'] = False

ax = sns.heatmap(matrix, xticklabels=genre_name, yticklabels=genre_name, annot=True,  cmap='RdYlGn_r')
ax

### 상위 100개의 단어들 중 중복되는 단어를 제외하고 상위 n(=15)개의 단어를 추출

In [14]:
n = 15
w1_, w2_ = [], []

for i in range(100):
    w1_.append(vectorizer.get_feature_names_out()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names_out()[w2[i][0]])

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break 

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break